In [ ]:
import pandas as pd
import numpy as np
import requests 
import re
import pylab
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from IPython.display import Image

# Data Gathering

### Twitter Archive

In [ ]:
archive_df= pd.read_csv('twitter-archive-enhanced.csv')


### Image Predictions

In [ ]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

filename = 'image-predictions.csv'
response = requests.get(url)
with open(filename, 'wb') as f:
    f.write(response.content)

In [ ]:
image_predictions_df = pd.read_csv('image-predictions.csv', sep='\t')

### Additional Data (Twitter API)

In [ ]:
api_df = pd.read_json('tweet-json.txt', lines = True)

# Data Assessing


## visual assesment

In [ ]:
archive_df.head()

In [ ]:
image_predictions_df.head()

In [ ]:
api_df.head()

## programmatic assessement

In [ ]:
archive_df.columns

In [ ]:
api_df.columns

In [ ]:
image_predictions_df.columns

In [ ]:
archive_df.info()

In [ ]:
api_df.info()
['possibly_sensitive', 'possibly_sensitive_appealable', 'quoted_status']

In [ ]:
api_df.describe()

In [ ]:
image_predictions_df.info()

In [ ]:
archive_df.shape

In [ ]:
api_df.shape

In [ ]:
image_predictions_df.shape

In [ ]:
archive_df.floofer.value_counts()

In [ ]:
archive_df.doggo.value_counts()

In [ ]:
archive_df.pupper.value_counts()

In [ ]:
archive_df.puppo.value_counts()

In [ ]:
archive_df.name.value_counts()

In [ ]:
archive_df.rating_numerator.value_counts()

In [ ]:
print(archive_df.loc[archive_df.rating_numerator ==1776, 'text'])
print(archive_df.loc[archive_df.rating_numerator ==666, 'text'])
print(archive_df.loc[archive_df.rating_numerator ==99, 'text'])

In [ ]:
#high rate dog
print(archive_df['text'][979]) 
#no picture, will be dropped later
print(archive_df['text'][189]) 
#9 dogs
print(archive_df['text'][1228]) 

In [ ]:
archive_df.rating_denominator.value_counts()

In [ ]:
print(archive_df.loc[archive_df.rating_denominator == 15, 'text']) 
print(archive_df.loc[archive_df.rating_denominator == 2, 'text']) 
print(archive_df.loc[archive_df.rating_denominator == 11, 'text']) 

In [ ]:
#tweet to explain rating
print(archive_df['text'][342]) 
#actual rating 9/10 need to change manually
print(archive_df['text'][2335]) 
#actual rating 14/10 need to be changed manually
print(archive_df['text'][784]) 
#retweet - it will be deleted when delete all retweets
print(archive_df['text'][1068]) 
#actual rating 10/10 need to be changed manually
print(archive_df['text'][1662]) 

In [ ]:
archive_df[archive_df.tweet_id.duplicated()]

In [ ]:
archive_df.describe()

In [ ]:
archive_df.isnull().sum()

In [ ]:
image_predictions_df.sample(1)

In [ ]:
# This is an image for tweet_id 856282028240666624
Image(url = 'https://pbs.twimg.com/media/Cvi2FiKWgAAif1u.jpg')

In [ ]:
image_predictions_df[image_predictions_df.tweet_id.duplicated()]

In [ ]:
image_predictions_df['p1'].value_counts()

In [ ]:
image_predictions_df['p2'].value_counts()

In [ ]:
image_predictions_df['p3'].value_counts()

## Clean

In [ ]:
new_api_df= api_df.copy()


##  Quality Issues
### api_df

- Contributors, coordinates, geo have no data at all, so we don't consider these fields.<br>
- extended_entities Without the data of this field, we can determine that the tweet does not have a picture.<br>
- in_reply_to_screen_name, in_reply_to_status_id, in_reply_to_status_id_str, in_reply_to_user_id, in_reply_to_user_id_str are the same group. <br>
- maybe_sensitive, possible_sensitive_appealable these fields have little to do with the theme.<br>
- place displays tweet location information, So this field is not retained.<br>
- quoted_status, quoted_status_id, quoted_status_id_str, quoted_status_permalink are the same group. When the message is a quoted tweet, these fields will record the relevant information, so they will be dropped.<br>
- \\retweeted_status: Retweets can be distinguished from typical Tweets by the existence of a retweeted_status attribute. So as long as it is forwarded data, we don't want it.<br>
- display_text_range indicates the range of text(DROP).<br>
- entities related information about the original tweets, because there is image prediction to find related image predictions, so this field is not used.<br>

- id_str type error, should be string object<br>
- is_quote_status: Indicates whether this is a Quoted Tweet. So as long as it is True we don't want
- lang type error, should be category datatype<br>
- source mixed html tag<br>
- retweeted are False, don't use this field<br>
- truncated are False, don't use this field<br>
- user is the same account @WeRateDogs, this field can also be removed<br>

## Tidiness issues
- merging three data sets in one<br>
- creating one column for dog stage<br>
- Standardize dog ratings as floats.<br>

- ### define

Remove unwanted fields, contributors, coordinates, geo, possibly_sensitive, possible_sensitive_appealable, place, display_text_range, entities, favorited, retweeted, truncated, user



- ### Code

In [ ]:
columns=['contributors', 'coordinates', 'geo', 'possibly_sensitive', 'possibly_sensitive_appealable', 'place', 'display_text_range', 'entities', 'favorited', 'retweeted', 'truncated', 'user']
new_api_df.drop(columns, axis=1, inplace=True)

- ### Test

In [ ]:
new_api_df.info()

- ### define
 Remove extended_entities NULL values

- ### Code

In [ ]:
new_api_df = new_api_df[new_api_df.extended_entities.notnull()]
new_api_df

- ### Test

In [ ]:
sum(new_api_df.extended_entities.isnull())


- ### define
Delete in_reply_to_screen_name, in_reply_to_status_id, in_reply_to_status_id_str, in_reply_to_user_id, in_reply_to_user_id_str 

Since these fields are information of the same group and in_reply_to_status_id with no data values.

In [ ]:
new_api_df = new_api_df[new_api_df.in_reply_to_status_id.isnull()]     

In [ ]:
print(new_api_df.in_reply_to_status_id.count(), 
      new_api_df.in_reply_to_user_id.count(),
      new_api_df.in_reply_to_screen_name.count(),
      new_api_df.in_reply_to_user_id.count(),
      new_api_df.in_reply_to_user_id_str.count())

- ### define
Delete data with is_quote_status to True<br>
Leave is_quote_status records as False.

- ### code

In [ ]:
new_api_df = new_api_df[~new_api_df.is_quote_status]
new_api_df

- ### Test


In [ ]:
sum(new_api_df.is_quote_status)

- ### define
Delete retweeded_status with valued data
Leave the retweeted_status method isnull() to True for data

- ### Code

In [ ]:
new_api_df = new_api_df[new_api_df.retweeted_status.isnull()]
new_api_df

- ### Test

In [ ]:
sum(new_api_df.retweeted_status.notnull())

- ### Define
Check the rest of the data and remove the fields without data.

In [ ]:
new_api_df.info()

- ### Code

In [ ]:
columns=['in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'retweeted_status', 'quoted_status_id', 'quoted_status_id_str', 'quoted_status']
new_api_df= new_api_df.drop(columns, axis=1)

- ### Test

In [ ]:
new_api_df.info()

- ### Define
Remove the is_quote_status, extended_entities field
- ### Code

In [ ]:
new_api_df.drop(['is_quote_status', 'extended_entities'], axis=1, inplace=True)

- ### Test

In [ ]:
new_api_df.info()

- ### Define

Correct id_str type is str datatype
- ### Code

In [ ]:
new_api_df.id_str = new_api_df.id_str.astype('str')

- ### Test

In [ ]:
new_api_df.id_str.dtype

- ### Define
Correct lang type is category datatype
- ### Code

In [ ]:
new_api_df.lang = new_api_df.lang.astype('category')

- ### Test

In [ ]:
new_api_df.lang.dtype

- ### Define

Trimming source content
- ### Code

In [ ]:
new_api_df.source.unique()

In [ ]:
replace = {
    '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>': 'iPhone',
    '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>': 'Web Client',
    '<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>': 'TweetDeck',
}
new_api_df.source.replace(replace, inplace=True)

- ### Test

In [ ]:
new_api_df.source.unique()

In [ ]:
new_api_df.head()

- ### Define
Merge the clean versions of  archive_df, image_predictions_df, and new_api_df dataframes.
- ### Code

In [ ]:
# Make a copy of the tables before merging and cleaning
archive_clean_df = archive_df.copy()
image_predictions_clean_df = image_predictions_df.copy()
api_clean_df = new_api_df.copy()


In [ ]:
dfs = pd.concat([archive_clean_df, image_predictions_clean_df, api_clean_df], join='outer', axis=1)

In [ ]:
dfs.head()

- ### Test

In [ ]:
dfs.info()

- ### Define
Create one column for the various dog types: doggo, floofer, pupper, pupp
- ### Code 

In [ ]:
# Extract the text from the columns into the new dog_type columnn
# handle none
dfs.doggo.replace('None', '', inplace=True)
dfs.floofer.replace('None', '', inplace=True)
dfs.pupper.replace('None', '', inplace=True)
dfs.puppo.replace('None', '', inplace=True)

# merge into column
dfs['dog_stage'] = dfs.doggo + dfs.floofer + dfs.pupper + dfs.puppo

# handle multiple stages
dfs.loc[dfs.dog_stage == 'doggopupper', 'dog_stage'] = 'doggo, pupper'
dfs.loc[dfs.dog_stage == 'doggopuppo', 'dog_stage'] = 'doggo, puppo'
dfs.loc[dfs.dog_stage == 'doggofloofer', 'dog_stage'] = 'doggo, floofer'

# handle missing values
dfs.loc[dfs.dog_stage == '', 'dog_stage'] = np.nan

- ### Test

In [ ]:
dfs[['dog_stage', 'doggo', 'floofer', 'pupper', 'puppo']].sample(10)

In [ ]:
dfs.info()

In [ ]:
dfs.dog_type.value_counts()

- ### Define
Delete retweets
- ### Code

In [ ]:
dfs = dfs[np.isnan(dfs.retweeted_status_id)]

- ### Test

In [ ]:
#Verify no non-null entires are left
dfs.info()

- ### Define
Remove columns no longer needed and duplicated columns.
- ### Code

In [ ]:
dfs = dfs.drop(['retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp','in_reply_to_status_id', 'in_reply_to_user_id', 'source', 'img_num', 'source', 'lang','created_at'], axis=1)

In [ ]:
dfs = dfs.loc[:,~dfs.columns.duplicated()]

- ### Test

In [ ]:
dfs.info()

- ### Define
Change the timestamp to correct datetime format
- ### Code

In [ ]:
#Remove the time zone from the 'timestamp' column
dfs['timestamp'] = dfs['timestamp'].str.slice(start=0, stop=-6)
# Change the 'timestamp' column to a datetime object
dfs['timestamp'] = pd.to_datetime(dfs['timestamp'], format = "%Y-%m-%d %H:%M:%S")

- ### Test

In [ ]:
dfs.head()

- ### Define
Correct naming issues
- ### Code

In [ ]:
dfs.name = dfs.name.str.replace('^[a-z]+', 'None')

- ### Test

In [ ]:
dfs['name'].value_counts()

- ### Define
Standardize dog ratings
- ### Code

In [ ]:
dfs['rating_numerator'] = dfs['rating_numerator'].astype(float)
dfs['rating_denominator'] = dfs['rating_denominator'].astype(float)

- ### Test

In [ ]:
dfs.info()

In [ ]:
# For loop to gather all text, indices, and ratings for tweets that contain a decimal in the numerator of the rating
decimals_text = []
decimals_index = []
decimals = []
for x, text in dfs['text'].iteritems():
    if bool(re.search('\d+\.\d+\/\d+', text)):
        decimals_text.append(text)
        decimals_index.append(x)
        decimals.append(re.search('\d+\.\d+', text).group())

In [ ]:
# Print ratings with decimals        
decimals_text

In [ ]:
# Print the indices of the ratings above (have decimal)
decimals_index

In [ ]:
decimals

In [ ]:
#Correctly converting the above decimal ratings to float
dfs.loc[decimals_index[0],'rating_numerator'] = float(decimals[0])
dfs.loc[decimals_index[1],'rating_numerator'] = float(decimals[1])
dfs.loc[decimals_index[2],'rating_numerator'] = float(decimals[2])
dfs.loc[decimals_index[3],'rating_numerator'] = float(decimals[3])

In [ ]:
dfs.loc[40]

In [ ]:
# Create a new column called rating, and calulate the value with new, standardized ratings
dfs['rating'] = dfs['rating_numerator'] / dfs['rating_denominator']
dfs.head()

- ### Define
Creating a new dog_breed column using the image prediction data
- ### Code


In [ ]:
dfs['dog_breed'] = 'None'
for i, row in dfs.iterrows():

    if row.p1_dog:
        dfs.at[i, 'dog_breed']= row.p1
    elif row.p2_dog and row.rating_numerator >= 10:
        dfs.at[i, 'dog_breed']= row.p2
    elif row.p3_dog and row.rating_numerator >= 10:
        dfs.at[i, 'dog_breed']= row.p3
    else:
        dfs.at[i, 'dog_breed']= 'None'

In [ ]:
dfs.dog_breed.value_counts()

In [ ]:
dfs.info()

# Storing

In [ ]:
dfs.to_csv('twitter_archive_master.csv', encoding='utf-8', index=False)

# Analyzing and Visualizations

- ### First Insight and Visualization<br>
Golden retriever is the most common dog in this dataset

In [ ]:
print(dfs['dog_breed'].value_counts())

In [ ]:
df_dog_breed = dfs.groupby('dog_breed').filter(lambda x: len(x) >= 30)

df_dog_breed['dog_breed'].value_counts().plot(kind = 'barh')
plt.title('Histogram of the Most Rated Dog Type')
plt.xlabel('Count')
plt.ylabel('Type of dog')

fig = plt.gcf() 
fig.savefig('output1.png',bbox_inches='tight');

- ### 2nd Insight and Visualization<br>
The highest ratings do not receive the most retweets.

In [ ]:
dfs.plot(x='retweet_count', y='rating', kind='scatter')
plt.xlabel('Retweet Counts')
plt.ylabel('Ratings')
plt.title('Retweet Counts by Ratings Scatter Plot')

fig = plt.gcf()
fig.savefig('output2.png',bbox_inches='tight');

- ### 3rd Insight <br>
    clumber has the lowest average rating Blenheim_spaniel has the highest average rating

In [ ]:
df_dog_type_mean = dfs.groupby('dog_breed').mean()

In [ ]:
df_dog_type_mean.head()

In [ ]:
df_dog_type_sorted = df_dog_type_mean['rating'].sort_values()

df_dog_type_sorted

- ### 4th Insight and Visualization<br>
    The most three common names are Charlie, Lucy and Cooper.       
       
 

In [ ]:
dfs.name.value_counts()[0:7].plot(kind ='barh', figsize=(15,8), title='Most Common Dog Names').set_xlabel("Number of Dogs");
fig = plt.gcf()
fig.savefig('output3.png',bbox_inches='tight');

In [ ]:
dfs.name.value_counts()